<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Feature Engineering</h4>
    <p style="font-size: 20px;">NBA API Data (2022-2024)</p>
</div>

<a name="Feature-Engineering"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

**[1. Create Team Matchups and Targets](#1.-Create-Team-Matchups-and-Targets)**

- [1.1. Clean Game Data](#1.1.-Clean-Game-Data)

- [1.2. Reshape to Game Matchups](#1.2.-Reshape-to-Game-Matchups)

- [1.3. Create Target Variables](#1.3.-Create-Target-Variables)

**[2. Create Rolling Window Statistics](#2.-Create-Rolling-Window-Statistics)**

# Setup

[Return to top](#Feature-Engineering)

In [2]:
pip install lime

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: matplotlib in c:\users\smcne\anaconda3\lib\site-packages (from lime) (3.5.1)



You should consider upgrading via the 'C:\Users\smcne\anaconda3\python.exe -m pip install --upgrade pip' command.


  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283837 sha256=e738273dc88e5b3536001f46d29b93457b8c7b40246896882d43b274ac69a3bf
  Stored in directory: c:\users\smcne\appdata\local\pip\cache\wheels\e6\a6\20\cc1e293fcdb67ede666fed293cb895395e7ecceb4467779546
Successfully built lime


In [3]:
import sys
from pathlib import Path
# get current working directory
cwd = %pwd
# add shared_code directory to Python sys.path
sys.path.append(str(Path(cwd).parent / "shared_code"))
# import all libraries in shared_code directory 'imports.py' file
from imports import *
%matplotlib inline

# Data

[Return to top](#Feature-Engineering)

In [102]:
adv_stats_df = pd.read_csv('../../data/original/nba_advanced_statistics_2021_2024.csv')
adv_stats_df.rename(columns={'minutes':'MIN'}, inplace=True)
adv_stats_df['MIN'] = adv_stats_df['MIN'].str.slice(0, 3)
adv_stats_df['MIN'] = adv_stats_df['MIN'].astype(int)

In [103]:
box_score_df = pd.read_csv('../../data/original/nba_games_box_scores_2022_2024.csv')

In [104]:
box_score_df.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7520,22023,1610612764,WAS,Washington Wizards,22300642,2024-01-27,WAS @ DET,W,240,118,45,100,0.450,11,34.0,0.324,17,21,0.810,16.0,34.0,50.0,26,10.0,4,9,19,14.0
7521,22023,1610612764,WAS,Washington Wizards,22300665,2024-01-29,WAS @ SAS,W,240,118,46,86,0.535,9,25.0,0.360,17,24,0.708,14.0,31.0,45.0,32,9.0,8,18,15,5.0
7522,22023,1610612764,WAS,Washington Wizards,22300676,2024-01-31,WAS vs. LAC,L,239,109,45,97,0.464,9,29.0,0.310,10,15,0.667,12.0,33.0,45.0,19,4.0,10,13,19,-16.0
7523,22023,1610612764,WAS,Washington Wizards,22300689,2024-02-02,WAS vs. MIA,L,239,102,37,90,0.411,11,42.0,0.262,17,21,0.810,6.0,37.0,43.0,28,5.0,4,8,25,-8.0
7524,22023,1610612764,WAS,Washington Wizards,22300705,2024-02-04,WAS vs. PHX,L,240,112,47,96,0.490,7,32.0,0.219,11,17,0.647,13.0,22.0,35.0,32,11.0,4,18,19,-28.0


In [105]:
adv_stats_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,MIN,estimatedOffensiveRating,offensiveRating,estimatedDefensiveRating,defensiveRating,estimatedNetRating,netRating,assistPercentage,assistToTurnover,assistRatio,offensiveReboundPercentage,defensiveReboundPercentage,reboundPercentage,estimatedTeamTurnoverPercentage,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE,TEAM_NAME,SEASON_ID,GAME_DATE,MATCHUP
0,22101221,1610612745,HOU,240,114.2,116.3,133.3,134.0,-19.0,-17.7,0.585,3.00,18.5,0.216,0.674,0.433,8.016,8.2,0.556,0.583,1.0,0.195,98.68,97.5,81.25,98.0,0.423,Houston Rockets,22021,2022-04-10,HOU vs. ATL
1,22101221,1610612737,ATL,240,133.3,134.0,114.2,116.3,19.0,17.7,0.644,2.23,20.8,0.326,0.784,0.567,13.325,13.4,0.638,0.666,1.0,0.199,98.68,97.5,81.25,97.0,0.577,Atlanta Hawks,22021,2022-04-10,ATL @ HOU
2,22101207,1610612748,MIA,240,116.1,116.5,110.8,112.4,5.3,4.1,0.500,1.18,16.5,0.167,0.646,0.440,17.468,17.5,0.632,0.670,1.0,0.199,97.84,97.0,80.83,97.0,0.543,Miami Heat,22021,2022-04-08,MIA vs. ATL
3,22101207,1610612737,ATL,240,110.8,112.4,116.1,116.5,-5.3,-4.1,0.405,1.13,13.2,0.354,0.833,0.560,15.250,15.5,0.523,0.566,1.0,0.199,97.84,97.0,80.83,97.0,0.457,Atlanta Hawks,22021,2022-04-08,ATL @ MIA
4,22101192,1610612737,ATL,240,118.8,121.6,104.2,104.0,14.6,17.6,0.605,4.33,19.6,0.200,0.773,0.468,6.039,6.2,0.554,0.588,1.0,0.201,99.10,98.0,81.67,97.0,0.548,Atlanta Hawks,22021,2022-04-06,ATL vs. WAS


<a name="1.-Create-Team-Matchups-and-Targets"></a>
# 1. Create Team Matchups and Targets

[Return to top](#Feature-Engineering)

<a name="1.1.-Clean-Game-Data"></a>
## 1.1. Clean Game Data

[Return to top](#Feature-Engineering)

We need to do three key things to clean the data:

1. Remove games with team aggregated game times of less than 238 minutes (which will remove exhibition matches).
2. Retain only games that are part of the regular season.
3. Remove any orphans (i.e., game IDs that do not have a partner) when reshaping to matchups.

Last 3 NBA regular seasons start and end dates:

- 2021-22 season: 2021-10-19 to 2022-04-10
- 2022-23 season: 2022-10-18 to 2023-04-09
- 2023-24 season: 2023-10-24 to 2024-04-14

In [106]:
# last 3 seasons start and end dates and labels
season_start_dates = ['2021-10-19', '2022-10-18', '2023-10-24']
season_end_dates   = ['2022-04-10', '2023-04-09', '2024-04-14']
season_labels      = ['2021-22', '2022-23', '2023-24']

In [107]:
# clean up the data
adv_stats_df_cleaned = utl.clean_team_bs_data(adv_stats_df, season_start_dates=season_start_dates, 
                                            season_end_dates=season_end_dates, season_labels=season_labels)

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 769 games


In [108]:
# clean up the data
box_score_df_cleaned = utl.clean_team_bs_data(box_score_df, season_start_dates=season_start_dates, 
                                            season_end_dates=season_end_dates, season_labels=season_labels)

Season 2021-22: 1230 games
Season 2022-23: 1230 games
Season 2023-24: 736 games


In [109]:
#get GAME_DATE, MATCHUP, GAME_ID, TEAM_ABBREVIATION fields from games_df
adv_stats_df_cleaned = pd.merge(adv_stats_df_cleaned, box_score_df_cleaned[['WL','PTS','GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION', 'PLUS_MINUS']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

adv_stats_df_cleaned.sort_values(by=['PTS'])

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,MIN,estimatedOffensiveRating,offensiveRating,estimatedDefensiveRating,defensiveRating,estimatedNetRating,netRating,assistPercentage,assistToTurnover,assistRatio,offensiveReboundPercentage,defensiveReboundPercentage,reboundPercentage,estimatedTeamTurnoverPercentage,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE,TEAM_NAME,SEASON_ID,GAME_DATE,MATCHUP,WL,PTS,PLUS_MINUS
225,22100717,1610612758,SAC,240,76.1,77.3,127.9,133.3,-51.8,-56.0,0.517,1.25,11.7,0.275,0.596,0.413,12.170,12.4,0.337,0.369,1.0,0.195,99.34,96.50,80.42,97.0,0.219,Sacramento Kings,2021-22,2022-01-25,SAC @ BOS,L,75,-53.0
890,22100075,1610612742,DAL,240,75.8,75.8,102.2,105.0,-26.3,-29.2,0.609,0.93,11.8,0.123,0.762,0.394,15.170,15.2,0.346,0.417,1.0,0.200,101.30,100.00,83.33,99.0,0.270,Dallas Mavericks,2021-22,2021-10-29,DAL @ DEN,L,75,-31.0
243,22100595,1610612752,NYK,240,84.7,88.2,114.5,115.1,-29.8,-26.9,0.448,0.93,11.5,0.296,0.714,0.479,15.801,16.5,0.437,0.438,1.0,0.195,87.54,85.50,71.25,85.0,0.291,New York Knicks,2021-22,2022-01-08,NYK @ BOS,L,75,-24.0
786,22100988,1610612742,DAL,240,82.8,83.7,109.7,116.3,-26.9,-32.6,0.407,1.38,9.5,0.250,0.580,0.400,8.602,8.7,0.349,0.397,1.0,0.200,95.26,92.00,76.67,92.0,0.224,Dallas Mavericks,2021-22,2022-03-09,DAL vs. NYK,L,77,-30.0
726,22100257,1610612741,CHI,240,80.5,81.9,109.1,114.7,-28.5,-32.8,0.419,1.44,11.4,0.140,0.667,0.389,9.414,9.6,0.400,0.420,1.0,0.201,97.76,94.50,78.75,94.0,0.279,Chicago Bulls,2021-22,2021-11-22,CHI vs. IND,L,77,-32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,22100899,1610612759,SAS,290,125.7,128.7,123.8,123.4,1.9,5.3,0.614,2.06,19.9,0.370,0.712,0.538,13.609,13.9,0.586,0.633,1.0,0.200,102.82,101.79,84.83,122.0,0.517,San Antonio Spurs,2021-22,2022-02-25,SAS @ WAS,W,157,4.0
4976,22300039,1610612754,IND,240,138.5,144.0,137.8,140.7,0.8,3.3,0.614,2.50,22.5,0.300,0.650,0.475,12.354,12.8,0.723,0.738,1.0,0.196,111.82,108.50,90.42,109.0,0.509,Indiana Pacers,2023-24,2023-11-21,IND @ ATL,W,157,5.0
2049,22100723,1610612766,CHA,240,140.4,147.7,114.6,117.8,25.8,29.9,0.685,2.85,23.5,0.333,0.577,0.468,11.554,12.1,0.710,0.735,1.0,0.197,111.24,107.00,89.17,107.0,0.608,Charlotte Hornets,2021-22,2022-01-26,CHA @ IND,W,158,32.0
3750,22200902,1610612746,LAC,290,132.7,133.6,136.0,135.4,-3.3,-1.8,0.661,1.56,21.9,0.238,0.720,0.500,18.962,19.1,0.735,0.769,1.0,0.197,108.10,108.00,90.00,131.0,0.495,LA Clippers,2022-23,2023-02-24,LAC vs. SAC,L,175,-1.0


<a name="1.2.-Reshape-to-Game-Matchups"></a>
## 1.2. Reshape to Game Matchups

[Return to top](#Feature-Engineering)

In [110]:
# identify non-stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP']

# reshape team box score data to wide format so each row is a game matchup
adv_stats_matchups_df = utl.reshape_team_bs_to_matchups(adv_stats_df_cleaned, non_stats_cols)

Season 2021-22: 1222 games
Season 2022-23: 1221 games
Season 2023-24: 728 games


In [111]:
adv_stats_matchups_df.head()

,GAME_ID,HOME_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_MIN,HOME_estimatedOffensiveRating,HOME_offensiveRating,HOME_estimatedDefensiveRating,HOME_defensiveRating,HOME_estimatedNetRating,HOME_netRating,HOME_assistPercentage,HOME_assistToTurnover,HOME_assistRatio,HOME_offensiveReboundPercentage,HOME_defensiveReboundPercentage,HOME_reboundPercentage,HOME_estimatedTeamTurnoverPercentage,HOME_turnoverRatio,HOME_effectiveFieldGoalPercentage,HOME_trueShootingPercentage,HOME_usagePercentage,HOME_estimatedUsagePercentage,HOME_estimatedPace,HOME_pace,HOME_pacePer40,...,AWAY_estimatedDefensiveRating,AWAY_defensiveRating,AWAY_estimatedNetRating,AWAY_netRating,AWAY_assistPercentage,AWAY_assistToTurnover,AWAY_assistRatio,AWAY_offensiveReboundPercentage,AWAY_defensiveReboundPercentage,AWAY_reboundPercentage,AWAY_estimatedTeamTurnoverPercentage,AWAY_turnoverRatio,AWAY_effectiveFieldGoalPercentage,AWAY_trueShootingPercentage,AWAY_usagePercentage,AWAY_estimatedUsagePercentage,AWAY_estimatedPace,AWAY_pace,AWAY_pacePer40,AWAY_possessions,AWAY_PIE,AWAY_TEAM_NAME,AWAY_WL,AWAY_PTS,AWAY_PLUS_MINUS
0,22101221,1610612745,HOU,240,114.2,116.3,133.3,134.0,-19.0,-17.7,0.585,3.00,18.5,0.216,0.674,0.433,8.016,8.2,0.556,0.583,1.0,0.195,98.68,97.5,81.25,...,114.2,116.3,19.0,17.7,0.644,2.23,20.8,0.326,0.784,0.567,13.325,13.4,0.638,0.666,1.0,0.199,98.68,97.5,81.25,97.0,0.577,Atlanta Hawks,W,130,16.0
1,22101207,1610612748,MIA,240,116.1,116.5,110.8,112.4,5.3,4.1,0.500,1.18,16.5,0.167,0.646,0.440,17.468,17.5,0.632,0.670,1.0,0.199,97.84,97.0,80.83,...,116.1,116.5,-5.3,-4.1,0.405,1.13,13.2,0.354,0.833,0.560,15.250,15.5,0.523,0.566,1.0,0.199,97.84,97.0,80.83,97.0,0.457,Atlanta Hawks,L,109,-4.0
2,22101192,1610612737,ATL,240,118.8,121.6,104.2,104.0,14.6,17.6,0.605,4.33,19.6,0.200,0.773,0.468,6.039,6.2,0.554,0.588,1.0,0.201,99.10,98.0,81.67,...,118.8,121.6,-14.6,-17.6,0.619,1.86,19.9,0.227,0.800,0.532,14.164,14.1,0.547,0.567,1.0,0.200,99.10,98.0,81.67,99.0,0.452,Washington Wizards,L,103,-15.0
3,22101182,1610612761,TOR,240,115.0,120.4,112.2,111.3,2.8,9.1,0.455,2.50,14.0,0.410,0.796,0.591,7.794,8.2,0.480,0.515,1.0,0.194,99.46,97.5,81.25,...,115.0,120.4,-2.8,-9.1,0.721,4.43,22.6,0.204,0.590,0.409,7.270,7.2,0.532,0.544,1.0,0.199,99.46,97.5,81.25,97.0,0.482,Atlanta Hawks,L,108,-10.0
4,22101163,1610612737,ATL,240,116.7,116.2,104.2,108.5,12.5,7.7,0.500,2.57,13.6,0.214,0.746,0.487,6.695,6.7,0.494,0.567,1.0,0.203,107.46,105.5,87.92,...,116.7,116.2,-12.5,-7.7,0.405,1.31,12.4,0.254,0.786,0.513,11.780,12.3,0.510,0.536,1.0,0.198,107.46,105.5,87.92,106.0,0.417,Brooklyn Nets,L,115,-7.0


<a name="1.3.-Create-Target-Variables"></a>
## 1.3. Create Target Variables

[Return to top](#Feature-Engineering)

There are three targets of interest:

1. **Total Game Points (over / under):** This can be calculated as the sum of `HOME_PTS + AWAY_PTS`.
2. **Difference in Game Points (plus / minus):** This can be calculated in relation to the home team as the following difference: `HOME_PTS - AWAY_PTS`.
3. **Game Winner (moneyline):** This can be defined in relation to the home team using the `HOME_WL` column, where a win for the home team is equal to 1 and a loss for the home team equal to 0. We will create a new column called `GAME_RESULT` for this indicator.

In [112]:
# create the above three target variables
adv_stats_matchups_df = utl.create_target_variables(adv_stats_matchups_df, 'HOME_WL', 'HOME_PTS', 'AWAY_PTS')

In [113]:
adv_stats_matchups_df[['GAME_DATE', 'GAME_ID',  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_PTS', 'AWAY_PTS', 'GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS']].tail()

,GAME_DATE,GAME_ID,HOME_TEAM_NAME,AWAY_TEAM_NAME,HOME_PTS,AWAY_PTS,GAME_RESULT,TOTAL_PTS,PLUS_MINUS
3166,2023-11-22,22300225,Charlotte Hornets,Washington Wizards,117,114,1,231,3.0
3167,2023-11-10,22300009,Washington Wizards,Charlotte Hornets,117,124,0,241,-7.0
3168,2023-11-08,22300157,Charlotte Hornets,Washington Wizards,116,132,0,248,-16.0
3169,2024-01-24,22300619,Detroit Pistons,Charlotte Hornets,113,106,1,219,7.0
3170,2023-10-27,22300077,Charlotte Hornets,Detroit Pistons,99,111,0,210,-12.0


<a name="2.-Create-Rolling-Window-Statistics"></a>
# 2. Create Rolling Window Statistics

[Return to top](#Feature-Engineering)

Here we create average box scores for each team over a rolling window of the previous $n$-games.

In [114]:
# identify stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM_ID', 'AWAY_TEAM_ID',
                  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_WL', 'AWAY_WL', 'HOME_MIN', 
                  'AWAY_MIN', 'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION']
stats_cols = [col for col in adv_stats_matchups_df.columns if col not in non_stats_cols]

In [125]:
# calculate rolling averages for each statistic and add them to the DataFrame
adv_stats_matchups_roll_df = utl.process_rolling_stats(
    adv_stats_matchups_df, 
    stats_cols, 
    target_cols=['GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS'],
    window_size=5,   # the number of games to include in the rolling window
    min_obs=1,       # the minimum number of observations present within the window to yield an aggregate value
    stratify_by_season=True,  # should the rolling calculations be reset at the start of each new season or be contiguous across seasons? 
    exclude_initial_games=0   # number of initial games to exclude from the rolling averages (optionally by season)
)

In [126]:
adv_stats_matchups_roll_df.tail()

,GAME_ID,GAME_RESULT,TOTAL_PTS,PLUS_MINUS,HOME_TEAM_NAME,SEASON_ID,GAME_DATE,ROLL_HOME_estimatedOffensiveRating,ROLL_HOME_offensiveRating,ROLL_HOME_estimatedDefensiveRating,ROLL_HOME_defensiveRating,ROLL_HOME_estimatedNetRating,ROLL_HOME_netRating,ROLL_HOME_assistPercentage,ROLL_HOME_assistToTurnover,ROLL_HOME_assistRatio,ROLL_HOME_offensiveReboundPercentage,ROLL_HOME_defensiveReboundPercentage,ROLL_HOME_reboundPercentage,ROLL_HOME_estimatedTeamTurnoverPercentage,ROLL_HOME_turnoverRatio,ROLL_HOME_effectiveFieldGoalPercentage,ROLL_HOME_trueShootingPercentage,ROLL_HOME_usagePercentage,ROLL_HOME_estimatedUsagePercentage,...,AWAY_TEAM_NAME,ROLL_AWAY_estimatedOffensiveRating,ROLL_AWAY_offensiveRating,ROLL_AWAY_estimatedDefensiveRating,ROLL_AWAY_defensiveRating,ROLL_AWAY_estimatedNetRating,ROLL_AWAY_netRating,ROLL_AWAY_assistPercentage,ROLL_AWAY_assistToTurnover,ROLL_AWAY_assistRatio,ROLL_AWAY_offensiveReboundPercentage,ROLL_AWAY_defensiveReboundPercentage,ROLL_AWAY_reboundPercentage,ROLL_AWAY_estimatedTeamTurnoverPercentage,ROLL_AWAY_turnoverRatio,ROLL_AWAY_effectiveFieldGoalPercentage,ROLL_AWAY_trueShootingPercentage,ROLL_AWAY_usagePercentage,ROLL_AWAY_estimatedUsagePercentage,ROLL_AWAY_estimatedPace,ROLL_AWAY_pace,ROLL_AWAY_pacePer40,ROLL_AWAY_possessions,ROLL_AWAY_PIE,ROLL_AWAY_PTS
2540,22300703,0,218,-16.0,San Antonio Spurs,2023-24,2024-02-03,109.40,111.06,111.00,111.82,-1.60,-0.76,0.725,1.854,21.26,0.351,0.730,0.537,16.763,17.02,0.541,0.568,1.0,0.197,...,Cleveland Cavaliers,115.80,115.72,103.10,104.58,12.70,11.14,0.623,2.022,19.64,0.223,0.756,0.505,13.187,13.18,0.582,0.606,1.0,0.197,100.604,99.9,83.252,100.2,0.555,115.6
3077,22300705,0,252,-28.0,Washington Wizards,2023-24,2024-02-04,102.90,104.78,114.96,116.44,-12.08,-11.68,0.716,2.286,20.26,0.228,0.679,0.463,12.991,13.20,0.506,0.542,1.0,0.198,...,Phoenix Suns,117.76,120.78,116.70,119.34,1.04,1.44,0.592,1.958,19.98,0.257,0.687,0.496,16.376,16.76,0.624,0.644,1.0,0.196,102.556,100.1,83.416,99.8,0.515,120.6
3017,22300704,0,210,-12.0,Detroit Pistons,2023-24,2024-02-04,112.80,114.90,115.68,116.66,-2.90,-1.74,0.604,2.160,18.60,0.268,0.731,0.502,12.510,12.76,0.547,0.579,1.0,0.199,...,Orlando Magic,112.70,112.82,108.88,111.72,3.80,1.10,0.587,1.790,17.66,0.293,0.757,0.508,13.589,13.58,0.534,0.572,1.0,0.199,99.600,98.3,81.916,98.4,0.496,111.0
3036,22300707,0,214,-18.6,Charlotte Hornets,2023-24,2024-02-04,103.18,106.10,121.30,123.40,-18.10,-17.32,0.644,2.106,18.78,0.239,0.737,0.474,12.814,13.16,0.514,0.542,1.0,0.198,...,Indiana Pacers,110.96,116.04,117.34,119.28,-6.38,-3.26,0.678,2.696,20.50,0.298,0.709,0.489,11.075,11.58,0.543,0.562,1.0,0.198,102.888,99.8,83.166,100.0,0.475,116.0
2492,22300706,1,222,40.0,Boston Celtics,2023-24,2024-02-04,116.58,118.74,112.84,114.92,3.72,3.86,0.594,3.020,18.78,0.272,0.702,0.494,11.042,11.28,0.564,0.588,1.0,0.198,...,Memphis Grizzlies,106.60,107.74,113.54,114.52,-6.92,-6.80,0.696,1.662,19.34,0.247,0.699,0.468,15.908,16.06,0.534,0.566,1.0,0.198,97.852,97.0,80.834,97.0,0.452,104.4


In [127]:
# write out the matchups with rolling features
adv_stats_matchups_roll_df.to_csv('../../data/processed/nba_team_matchups_rolling_adv_stats_2021_2024_r05.csv', index=False)